In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用自定义容器进行在线预测的自定义训练文本二元分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用 Python 的 Vertex 客户端库，通过自定义容器进行训练，并部署一个用于在线预测的自定义文本二分类模型。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)数据集。在本教程中使用的数据集版本存储在一个公共的云存储桶中。训练好的模型可以预测评论是积极的还是消极的情感。

### 目标

在本教程中，您将使用Vertex客户端库在自定义Docker容器中从Python脚本创建一个自定义模型，并通过发送数据对部署的模型进行预测。您也可以选择使用`gcloud`命令行工具或通过Google Cloud Console在线创建自定义模型。

执行的步骤包括：

- 创建用于训练模型的Vertex自定义作业。
- 使用自定义容器训练一个TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为一个Vertex `Model`资源。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

费用

本教程使用 Google Cloud（GCP）的计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果有这个选项，确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 300 美元的免费信用，可用于支付计算/存储成本。

2. [确保启用您的项目的计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分。 以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持信息，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行通过oAuth身份验证。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务账户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

使用 Vertex 客户端库提交自定义训练作业时，您需要将包含您的训练代码的 Python 包上传到一个云存储桶中。Vertex 将从这个包中运行代码。在本教程中，Vertex 还会将您作业的训练模型保存在相同的桶中。然后，您可以基于此输出创建一个 `Endpoint` 资源，以便提供在线预测。

在下方设置您的云存储桶的名称。桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时，运行以下单元格以创建您的 Cloud Storage 存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入 Vertex 客户端库

将 Vertex 客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和终端服务终点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和终端资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置硬件加速器（如 GPU）用于训练和预测。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 使用支持 GPU 的容器映像和分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 分配给每个 VM 的 GPU 容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

对于 GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 以在 CPU 上运行容器映像。

*注意*：TF 2.3 之前的版本对于 GPU 支持将无法加载本教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已经修复了这个问题，这是由于在服务功能中生成的静态图操作造成的。如果在您自己的自定义模型上遇到这个问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）映像

接下来，我们将为预测设置Docker容器映像

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`代表版本2.1，`1-15`代表版本1.15。以下列出了一些可用的预构建映像：

  - TensorFlow 1.15
    - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
  - TensorFlow 2.1
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
  - TensorFlow 2.2
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
  - TensorFlow 2.3
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
  - XGBoost
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
  - Scikit-learn
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 的数量

*注意：以下不支持用于训练:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 和 8 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始创建自己的自定义模型和训练IMDB电影评论了。

设置客户端

Vertex客户端库工作方式类似于客户端/服务器模型。在您的一侧（Python脚本）中，您将创建一个客户端，该客户端将向Vertex服务器发送请求并接收响应。

在本教程中的不同步骤中，您将使用不同的客户端。因此，请提前设置它们。

- 用于`Model`资源的模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练一个模型

有两种方法可以使用容器镜像来训练一个自定义模型：

- **使用 Google Cloud 预构建的容器**。如果使用预构建的容器，您还需要指定要安装到容器镜像中的 Python 包。这个 Python 包包含了您用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用您自己的容器，容器需要包含您用于训练自定义模型的代码。

### 创建一个Docker文件

在本教程中，您将使用自己的自定义容器来训练一个IMDB电影评论模型。

为了使用自己的自定义容器，您需要创建一个Docker文件。首先，您将创建一个用于容器组件的目录。

### 检查培训套餐

#### 套餐布局

在开始培训之前，您将查看如何为自定义培训工作编制Python包。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境的指令。

文件`trainer/task.py`是执行自定义培训工作的Python脚本。*注意*，当我们在工作池规范中引用它时，我们会用一个点(`trainer.task`)替换目录斜杠，并删除文件后缀(`.py`)。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: IMDB Movie Reviews text binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

任务.py 内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我不会详细介绍，只是让您浏览。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）加载 IMDB 电影评论数据集。
- 使用 TF.Keras 模型 API 构建一个简单的 RNN 模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用指定的 epoch 训练模型（`fit()`）。
- 将已训练的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for IMDB

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_rnn_model(encoder)

# Train the model
model.fit(train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

#### 编写Docker文件内容

将代码容器化的第一步是创建一个Docker文件。 在你的Docker文件里，你会包括运行容器镜像所需的所有命令。 它将安装你正在使用的所有库，并为你的训练代码设置入口点。

1. 安装来自TensorFlow仓库的预定义容器镜像，用于深度学习图像。
2. 复制Python训练代码，稍后会展示。
3. 将入口点设置为Python训练脚本`trainer/task.py`。 注意，ENTRYPOINT命令中不需要包含`.py`，因为它是隐含的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-1
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

#### 本地构建容器

接下来，您将为您的客户容器提供一个名称，在将其提交到Google容器注册中时将使用这个名称。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/imdb:v1"

接下来，建造容器。

In [ ]:
! docker build custom -t $TRAIN_IMAGE

#### 在本地测试容器

在您的笔记本实例中运行容器，以确保它可以正常工作。您将运行它5次。

In [ ]:
! docker run $TRAIN_IMAGE --epochs=5

注册自定义容器

当您完成在本地运行容器后，请将其推送到 Google 容器注册表。

In [ ]:
! docker push $TRAIN_IMAGE

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

准备您的定制工作规范

现在您的客户已经准备好了，您的第一步是为您的定制培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：您将用于培训的机器类型和数量的规范（单一或分布式）
- `container_spec`：定制容器的规范。

### 准备您的机器规格

现在为您的自定义训练工作定义机器规格。这告诉 Vertex 需要为训练提供什么类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则表示您使用 GPU；否则您将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义用于自定义训练作业的磁盘规格。这会告诉Vertex每台机器实例上为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：可选SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 准备您的容器规范

现在为您的自定义训练容器定义容器规范：

- `image_uri`：自定义容器镜像。
- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数。

In [ ]:
JOB_NAME = "_custom_container" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

container_spec = {
    "image_uri": TRAIN_IMAGE,
    "args": CMDARGS,
}

### 定义工作池规范

接下来，您需要为您的自定义训练作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要部署的此机器类型的实例数。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。
- `container_spec`：要在VM实例上安装的Docker容器。

In [ ]:
worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
        "disk_spec": disk_spec,
    }
]

### 组装工作规范

现在组装自定义工作规范的完整描述：

- `display_name`：您分配给此自定义工作的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器VM实例的规范。
    - `base_output_directory`：这告诉服务云存储位置，可以在其中保存模型工件（当变量`DIRECT = False`时）。然后，服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

-`custom_job`：自定义作业的规范。

该辅助函数调用作业客户端服务的`create_custom_job`方法，并使用以下参数：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将在`response`对象中显示返回的一些字段，最感兴趣的两个字段是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您可以保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`: 自定义作业的 Vertex 完全限定标识符。

这个辅助函数调用作业客户端服务的 `get_custom_job` 方法，带有以下参数：

- `name`: 自定义作业的 Vertex 完全限定标识符。

如果您还记得，当您调用 `create_custom_job` 方法时，在 `response.name` 字段中获得了自定义作业的 Vertex 完全限定标识符，并将其保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将“结束时间”减去“开始时间”来计算训练模型实际花费的时间。对于您的模型，我们需要知道保存模型的位置，在Python脚本中将保存在您的本地云存储桶中的“MODEL_DIR + '/saved_model.pb'”。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras `model.load_model()` 方法，传递保存模型的 Cloud Storage 路径 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来看看模型的表现如何。

### 加载评估数据

您将使用`load()`方法从`tfds.datasets`加载IMDB电影评论测试（留存）数据。这将返回一个包含两个元素的元组。第一个元素是数据集，第二个是关于数据集的信息，其中包含预定义的词汇编码器。编码器将单词转换为数值嵌入，这是预训练并在自定义训练脚本中使用的。

当您训练模型时，需要为文本设置一个固定的输入长度。对应的`tf.dataset`的`padded_batch()`属性被设置为将每个输入序列填充到相同的形状以进行批处理。

对于测试数据，您还需要相应地设置`padded_batch()`属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 执行模型评估

现在评估自定义工作中的模型表现如何。

In [ ]:
model.evaluate(test_dataset)

##上传模型进行服务

接下来，您将从自定义作业中上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型期望的格式。如果您将编码数据发送到Vertex，您的服务函数将确保数据在传递给模型之前在模型服务器上进行解码。

###服务函数如何工作

当您向在线预测服务器发送请求时，请求会被一个HTTP服务器接收。HTTP服务器会从HTTP请求内容主体中提取出预测请求。提取出的预测请求会被转发给服务函数。对于Google预构建的预测容器，请求内容会作为`tf.string`传递给服务函数。

服务函数由两部分组成:

- `预处理函数`:
  - 将输入（`tf.string`）转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型时相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数都会被转换为静态图，然后与模型融合。来自底层模型的输出会传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，一个需要考虑的问题是它们作为静态图运行。这意味着您不能使用动态图需要的TF图操作。如果您这样做，您将在服务函数的编译过程中收到一个错误提示，指出您正在使用不受支持的EagerTensor。

获取服务功能签名

您可以通过将模型重新加载到内存中，并查询每个层对应的签名来获取您模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要了解服务功能的输入层名称 -- 这将在稍后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，该模型以SavedModel格式存储，上传到`Model`服务中，该服务将为您的模型实例化一个Vertex `Model`资源实例。完成后，您可以像使用任何其他Vertex `Model`资源实例一样使用这个`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的人类可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：指向我们的SavedModel艺术品的Cloud Storage路径。在本教程中，这是`trainer/task.py`保存模型构件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了它。

辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`，`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个由以下字段组成的字典对象：

- `display_name`：`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，您会将其留空(`''`)。
- `artificat_uri`：保存模型的Cloud Storage路径，以SavedModel格式存储。
- `container_spec`：这是要安装在`Endpoint`资源上的Docker容器的规范，`Model`资源将从中提供预测。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源返回一个长时间运行的操作，因为可能需要一些时间。您调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时会返回。

辅助函数返回对应的Vertex Model实例的Vertex全限定符标识符`upload_model_response.model`。您将保存这个标识符以供后续步骤使用，存储在变量`model_to_deploy_id`中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model("imdb-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

获取`Model`资源信息

现在让我们只获取您的模型的模型信息。使用这个辅助函数`get_model`，传入以下参数：

- `name`: `Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex `Model`客户端服务的`get_model`方法，传入以下参数：

- `name`: `Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练好的Vertex自定义`Model`资源。这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个端点，用于部署模型以便提供预测，需要传入以下参数：

- `display_name`: 用于`Endpoint`资源的可读名称。

这个辅助函数使用端点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`: 用于`Endpoint`资源的可读名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来提供用于服务的`Endpoint`资源。您需要调用`response.result()`，这是一个同步调用，当`Endpoint`资源准备好时会返回。这个辅助函数返回`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "imdb_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端节点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您有几种选择可以扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例上时，将会配置固定数量的计算实例，并且在线预测请求将在它们之间均匀分配。

- 自动扩展：在线预测请求在可扩展数量的计算实例上分配。
  - 在模型首次部署时设置计算实例的最小（`MIN_NODES`）数量进行配置和去配置，并且根据负载情况设置计算实例的最大（`MAX_NODES`）数量进行配置。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署`Model`资源到`Endpoint`资源

使用这个辅助函数`deploy_model`，将`Model`资源部署到您为提供预测创建的`Endpoint`资源，使用以下参数：

- `model`：要从训练管道上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的 Vertex 完全限定端点标识符。

辅助函数调用`Endpoint`客户端服务的方法`deploy_model`，该方法接受以下参数：

- `endpoint`：要将`Model`资源部署到的 Vertex 完全限定`Endpoint`资源标识符。
- `deployed_model`：部署模型的要求规范。
- `traffic_split`：流量分配给该模型的端点的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为**{ "0": 100 }**，其中"0"指的是上传该模型，100表示流量的百分之百。
   - 如果端点上有现有模型，流量将被分割，使用`model_id`指定为**{ "0": 百分比, model_id: 百分比, ... }**，其中`model_id`是要部署到端点的现有模型的模型标识符。百分比必须加起来等于100。

现在让我们更深入地讨论`deployed_model`参数。该参数被指定为一个具有最小必需字段的 Python 字典：

- `model`：要部署（上传）的模型的 Vertex 完全限定模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：禁用容器事件的记录，例如执行失败（默认情况下启用容器记录）。在调试部署时通常启用容器日志记录，然后在生产部署时禁用。
- `dedicated_resources`：指的是为提供预测请求而扩展的计算实例（副本）的数量。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量`DEPLOY_GPU != None`来使用 GPU；否则只会分配 CPU。
  - `min_replica_count`：最初提供的计算实例数量，您之前设置为变量`MIN_NODES`。
  - `max_replica_count`：缩放到的最大计算实例数量，您之前设置为变量`MAX_NODES`。

#### 流量分配

现在让我们更深入地讨论`traffic_split`参数。该参数被指定为一个Python字典。起初可能会有点令人困惑。让我解释一下，您可以将多个模型实例部署到一个端点，然后设置每个实例的流量（百分比）。

为什么要这样做？也许您已经在生产环境中部署了以前的版本--让我们称其为v1。在v2上得到了更好的模型评估结果，但在部署到生产环境之前，您并不确定是否真的更好。因此，在流量分配的情况下，您可能希望将v2部署到与v1相同的端点，但只占流量的百分之十。这样，您可以在不扰乱大多数用户的情况下监视其运行情况，直到最终做出决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`同步等待操作完成，这将阻塞直到模型部署。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的部署。

In [ ]:
DEPLOYED_NAME = "imdb_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

进行在线预测请求

现在对你部署的模型进行在线预测。

### 准备请求内容

由于数据集是 `tf.dataset`，它充当生成器的角色，我们必须将其用作迭代器来访问测试数据中的数据项。 我们通过以下方式从测试数据中获取单个数据项：

- 使用 `take(1)` 方法将每次迭代要绘制的批次数量设置为一。
- 通过测试数据进行一次迭代 -- 也就是在for循环中使用 break。
- 在单次迭代中，我们保存以元组形式表示的数据项。
- 数据项将是元组的第一个元素，然后您将其从张量转换为numpy数组 -- `data[0].numpy()`。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]

test_dataset.take(1)
for data in test_dataset:
    print(data)
    break

test_item = data[0].numpy()

### 发送预测请求

好的，现在您有一个测试数据项。使用这个辅助函数`predict_data`，它接受以下参数：

- `data`：测试数据项是一个填充为64的numpy 1D数组。
- `endpoint`：模型部署的Vertex AI完全限定标识符。
- `parameters_dict`：用于服务的附加参数。

这个函数使用预测客户端服务，并调用`predict`方法，带有以下参数：

- `endpoint`：模型部署的Vertex AI完全限定标识符。
- `instances`：要预测的实例（数据项）列表。
- `parameters`：用于服务的附加参数。

要将测试数据传递给预测服务，您必须将其打包以传输到服务二进制文件，操作步骤如下：

    1. 将数据项从1D numpy数组转换为1D Python列表。
    2. 将预测请求转换为序列化的Google protobuf (`json_format.ParseDict()`)

预测请求中的每个实例是一个字典条目，格式如下：

                        {input_name: content}

- `input_name`：底层模型的输入层的名称。
- `content`：数据项作为1D Python列表。

由于`predict()`服务可以接受多个数据项（实例），您将将单个数据项作为一个数据项的列表发送。最后一步，您将实例列表打包到Google的protobuf格式中，这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，其中列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到：

- `predictions` -- 在0（负面）和1（正面）之间的预测二进制情绪。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 取消部署 `Model` 资源

现在从正在提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到时由终结点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

此函数调用终结点客户端服务的 `undeploy_model` 方法，使用以下参数：

- `deployed_model_id`：当 `Model` 资源被部署时终结点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在剩余的部署模型之间分配流量在 `Endpoint` 资源上。

由于这是在 `Endpoint` 资源上唯一部署的模型，您可以简单地通过将其设置为 {} 来使 `traffic_split` 为空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调优作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME